# E.D.A.: CORRELATION BETWEEN  UNEMPLOYMENT & SUICIDE RATES IN 43 COUNTRIES FROM 2000 TO 2016  
By Daniel Del Valle González  2020-2021

## STEP 1 - DATA CLEANING & NORMALISING
## STEP 2 - DATA ANALISYS & TENDENCIES. TOP & BOTTOM COUNTRIES IN SUICIDE & UNEMPLOYMENT RATES
## STEP 3 - GRAPHIC VISUALISATION OF EACH DATAFRAME TENDENCIES

## STEP 1 - DATA CLEANING & NORMALISING

### We got 2 csv from:   
1 - https://www.kaggle.com/szamil/who-suicide-statistics csv for suicides from 2000.  
2 - https://stats.oecd.org/Index.aspx?QueryId=64198# csv for unemployment rates. I selected and added some countries that by defect were not included.  

We create 2 DataFrames from them:

In [ ]:
import pandas as pd
import numpy as np
from utils.folders_tb import *
from utils.mining_data_tb import *
from utils.visualization_tb import *
from utils.Unemployment import *
from utils.Suicide import *


In [ ]:
suicide = pd.read_csv("..\\documentation\\who_suicide_statistics.csv")
infotizer(suicide)

In [ ]:
unemployment = pd.read_csv("..\\documentation\\unemployment_all_ratio.csv")
infotizer(unemployment)

## Both had to be cleaned and "synchronised" in many ways:

### Selecting values only for countries in both dataframes  (a total of 43 countries do have data in both dataframes)

In [ ]:
suic = suicide.loc[suicide['country'].isin(countrylist)]
unemp = unemployment.loc[unemployment["Country"].isin(countrylist)] 

### Unemployment CSV also included "employment rate" and other values (columns). We only select those regarding to "unemployment rate"

In [ ]:
unemp = unemp.loc[unemp['Series'] == "Unemployment rate"]

### Unemployment DataFrame had some groups of countries that did not appear on the Suicides Rates DataFrame, so are discarded.

In [ ]:
unemp = unemp[~unemp.Country.str.contains("Euro" or "OECD", na=False)]                                    #discarding groups of countries(no info in the other dataframe)

### Unemployment DataFrame columns are to be renamed with similar names than those to be contrasted with in Suicide DataFrame

In [ ]:
unemp = unemployment.rename(columns={'SEX': 'Gender', 'Value':'Unemploy_Rate', "Time": "Year"}) #renaming columns for better understanding and according to the other dataframe
suic = suic.rename(columns={'sex': 'gender'}) 

unemp = unemp[['Country', 'Gender', 'Age', 'Year', 'Unemploy_Rate']]                            #selecting only needed columns

In [ ]:
column_lower(suic)   
column_lower(unemp)                                                                             #normalising columns names

### Dataframes had data in different years ranges, so we had to cut them to fit:

In [ ]:
unemp = unemp[unemp['year'] <= 2016]
suic = suic[suic['year'] >= 2000]

### On the "gender" columns there was a "MW" value (ambiguous, guessed to mean Men+ Women). It was discarded. Gender values are normalised to fit with the other DataFrame

In [ ]:
unemp.drop(unemp[unemp.gender == 'MW'].index, inplace=True)
unemp.loc[unemp['gender'] == "MEN", 'gender'] = "male"                                                             #normalising "gender" column formats
unemp.loc[unemp['gender'] == "WOMEN", 'gender'] = "female"          

### On the "age" columns the ranges where different (fewer and broader in Suicides Df, narrower in Unemployment Df. Synchronisation was not an easy task)

In [ ]:
suic['age'] = suic['age'].str.rstrip('years ')                                                                     #normalising age formats
unemp['age'] = unemp['age'].str.replace(' to ', '-')                                             

unemp = unemp[(unemp['age'] == '15-24') | (unemp['age'] == '25-34') |(unemp['age'] == '35-44') |(unemp['age'] == '45-54') | (unemp['age'] == '55-64') |(unemp['age'] == '65-69') | (unemp['age'] == '70-74')]                                                                                               #discarding duplicated age ranges(some are include inside others) 

unemp.loc[(unemp["age"] == '35-44') | (unemp["age"] == '45-54'), 'age'] = '35-54'                                  #merging smaller ranges into a bigg one (and common with my other csv)
unemp.loc[(unemp["age"] == '55-64') | (unemp["age"] == '65-69') | (unemp["age"] == '70-74'), 'age'] = "55-74"

## STEP 2 - DATA ANALISYS & TENDENCIES. TOP & BOTTOM COUNTRIES IN SUICIDE & UNEMPLOYMENT RATES

### Suicide is stadistically more common visualised in "suicides rate per 100.000 people", so we create a column regarding that measure:

In [ ]:
suic["suic_100k"] = ((suic.suicides_no/suic.population)*100000).round(2)              #only 2 decimals is enough

### Then we create some sub-dataframes with info centered in Age: 

In [ ]:
aggregation_functions = {'suic_100k': 'mean'}                                       #creating agg functions to make it faster when grouping tables
aggregation_functions2 = {'unemploy_rate': 'mean'}

In [ ]:
suic_ages_mean = suic.groupby(suic['age']).aggregate(aggregation_functions).sort_values("suic_100k",ascending=False).round(2)
suic_ages_mean

In [ ]:
unemp_ages_mean = unemp.groupby('age').aggregate(aggregation_functions2).sort_values("unemploy_rate", ascending=False).round(2)
unemp_ages_mean

### Also we create some sub-dataframes with info centered in distribution per Country:

In [ ]:
suic_countries_mean = suic.groupby('country').aggregate(aggregation_functions).sort_values("suic_100k",ascending=True).round(2)
suic_countries_mean

In [ ]:
unemp_countries_mean = unemp.groupby('country').aggregate(aggregation_functions2).sort_values("unemploy_rate", ascending=False).round(2)
unemp_countries_mean

In [ ]:
most_per_100k = suic_countries_mean.sort_values("suic_100k", ascending=False).round(2).head()
least_per_100k = suic_countries_mean.sort_values("suic_100k", ascending=False).round(2).tail()
print(most_per_100k)
print(least_per_100k)

In [ ]:
most_of_all = suic.groupby("country").sum().sort_values("suicides_no", ascending=False).head()                #top and bottom countries for absolute number of suicides
least_of_all = suic.groupby("country").sum().sort_values("suicides_no", ascending=False).tail()
print(most_of_all)
print(least_of_all)

In [ ]:
most_unemp = unemp_countries_mean.sort_values("unemploy_rate", ascending=False).round(2).head(5)               #top and bottom countries for unemployment rate
least_unemp = unemp_countries_mean.sort_values("unemploy_rate", ascending=False).round(2).tail(5)
print(most_unemp)
print(least_unemp)

### Suicide countries of Interest:

In [ ]:
suic_pivot = pd.pivot_table(suic, index = ['country', 'year'], values = ['suicides_no', 'suic_100k']).round(2)

In [ ]:
lt_su = suic_pivot.loc["Lithuania"]
lt_su

In [ ]:
ru_su = suic_pivot.loc["Russian Federation"]
ru_su

In [ ]:
sa_su = suic_pivot.loc["South Africa"]
sa_su

### Unemployment countries of Interest:

In [ ]:
unemp_pivot_mean_gndr = pd.pivot_table(unemp, index = ['country', 'year', 'gender'], values = ['unemploy_rate']).round(2)
unemp_pivot_mean_gndr

In [ ]:
sp_un = unemp_pivot_mean_gndr.loc["Spain"]                              #unemployment evolution in Spain per gender : tendency is to equal
sp_un

In [ ]:
unemp_pivot_mean = pd.pivot_table(unemp, index = ['country', 'year'], values = ['unemploy_rate']).round(2)
unemp_pivot_mean

In [ ]:
nw_un = unemp_pivot_mean.loc["Norway"]
nw_un

In [ ]:
sa_un = unemp_pivot_mean.loc["South Africa"]
sa_un

## STEP 3 - GRAPHIC VISUALISATION OF EACH DATAFRAME TENDENCIES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import psutil
import plotly.io as pio
from Unemployment import *
from Suicide import *